<a href="https://colab.research.google.com/github/Komsomolochka/stats-for-data-analysis/blob/main/churn_analysis/churn_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [191]:
import pandas as pd
import numpy as np
from scipy import stats
from itertools import combinations
from statsmodels.stats.proportion import proportion_confint
from statsmodels.sandbox.stats.multicomp import multipletests

In [3]:
data = pd.read_csv('/content/churn_analysis.csv')

In [25]:
data.head()

,Unnamed: 0,state,account_length,area_code,intl_plan,vmail_plan,vmail_message,day_mins,day_calls,day_charge,eve_mins,eve_calls,eve_charge,night_mins,night_calls,night_charge,intl_mins,intl_calls,intl_charge,custserv_calls,treatment,mes_estim,churn
0,0,KS,128,415,no,yes,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1,1,0.65,False.
1,1,OH,107,415,no,yes,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1,0,0.55,False.
2,2,NJ,137,415,no,no,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0,0,0.72,False.
3,3,OH,84,408,yes,no,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2,1,0.28,False.
4,4,OK,75,415,yes,no,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3,2,0.45,False.


Давайте рассмотрим всех пользователей из контрольной группы (treatment = 1). Для таких пользователей мы хотим проверить гипотезу о том, что штат абонента не влияет на то, перестанет ли абонент пользоваться услугами оператора. 

Для этого мы воспользуемся критерием хи-квадрат.  Постройте таблицы сопряженности между каждой из всех 1275 возможных неупорядоченных пар штатов и значением признака churn.  Для каждой такой таблицы 2x2 применить критерий хи-квадрат можно с помощью функции 
scipy.stats.chi2_contingency(subtable, correction=False)

In [30]:
state_churn = pd.crosstab(index=data[data['treatment']==1]['state'], columns=data[data['treatment']==1]['churn'])

In [34]:
chi2_cross_table = pd.DataFrame(columns=['state_1', 'state_2', 'p-value'])

In [38]:
i=0
for state1, state2 in combinations(state_churn.index, 2):
    chi2_ = state_churn.loc[[state1, state2],:]
    chi2_cross_table.loc[i, 'state_1'] = state1
    chi2_cross_table.loc[i, 'state_2'] = state2
    chi2_cross_table.loc[i, 'p-value'] = stats.chi2_contingency(chi2_.values, correction=False)[1]
    i+=1

In [40]:
chi2_cross_table.head()

,state_1,state_2,p-value
0,AK,AL,0.213621
1,AK,AR,0.0357288
2,AK,AZ,0.517397
3,AK,CA,0.0277369
4,AK,CO,0.101066


Заметьте, что, например, (AZ, HI)  и  (HI, AZ) — это одна и та же пара. Обязательно выставьте correction=False (о том, что это значит, вы узнаете из следующих вопросов).

Сколько  достигаемых уровней значимости оказались меньше, чем α=0.05?

In [43]:
chi2_cross_table[chi2_cross_table['p-value']<0.05]['p-value'].count()

34

*Ответ: 34*

*Интерпретация числа достигаемых уровней значимости, меньших α=0.05, некорректна, поскольку не сделана поправка на множественную проверку гипотез.*


*Применение критерия xи-квадрат для этих данных не обосновано, потому что не выполняются условия, при которых этот критерий дает правильные результаты.*

В основе критерия xи-квадрат лежит предположение о том, что если верна нулевая гипотеза, то дискретное биномиальное распределение данных по клеткам в  таблице сопряженности  может быть аппроксимировано с помощью непрерывного распределения xи-квадрат.  Однако точность такой аппроксимации существенно зависит от суммарного количества наблюдений и их распределения в этой таблице (отсюда и ограничения при использовании критерия xи-квадрат).

Одним из способов коррекции точности аппроксимации является поправка Йетса на непрерывность.  Эта поправка заключается в  вычитании константы 0.5  из каждого модуля разности наблюденного  O_i и ожидаемого  E_i значений.

Такая поправка, как несложно догадаться по формуле, как правило, уменьшает  значение статистики  chi2, то есть увеличивает достигаемый уровень значимости.

Эта поправка обычно используется для таблиц сопряженности размером 2x2  и для небольшого количества наблюдений. Такая поправка, однако, не является серебрянной пулей, и часто критикуется за то, что статистический критерий при ее использовании становится слишком консервативным, то есть часто не отвергает нулевую гипотезу там, где она неверна (совершает ошибку II рода). 

Полезно знать, что эта поправка часто включена по  умолчанию (например, в функции scipy.stats.chi2_contingency) и понимать ее влияние на оценку достигаемого уровня значимости. 

Проведите те же самые сравнения, что и в вопросе №1, только с включенной коррекцией 

In [55]:
chi2_cross_table_corr = pd.DataFrame(columns=['state_1', 'state_2', 'p-value'])

In [56]:
i=0
for state1, state2 in combinations(state_churn.index, 2):
    chi2_ = state_churn.loc[[state1, state2],:]
    chi2_cross_table_corr.loc[i, 'state_1'] = state1
    chi2_cross_table_corr.loc[i, 'state_2'] = state2
    chi2_cross_table_corr.loc[i, 'p-value'] = stats.chi2_contingency(chi2_.values, correction=True)[1]
    i+=1

In [58]:
chi2_cross_table_corr[chi2_cross_table_corr['p-value']<0.05]['p-value'].count()

0

*Достигаемые уровни значимости на наших данных, полученные с помощью критерия xи-квадрат с поправкой Йетса, в среднем получаются больше, чем соответствующие значения без поправки.*

*Количество достигаемых уровней значимости, меньших, чем 0.05, в точности равно нулю. То есть поправка увеличила достигаемые уровни значимости настолько, что больше ни одно из значений достигаемого уровня значимости не попадает в диапазон от 0 до 0.05.*

Что если у нас мало данных,  мы не хотим использовать аппроксимацию дискретного распределения непрерывным и использовать сомнительную поправку,  предположения  критерия xи-квадрат не выполняются, а проверить гипотезу о том, что данные принадлежат одному распределению, нужно ?

В таком случае прибегают к так называемому точному критерию Фишера. Этот  критерий не использует приближений  и в точности вычисляет значение достигаемого уровня значимости используя комбинаторный подход.

Посчитайте для каждой пары штатов, как и в первом задании, достигаемый уровень значимости с помощью точного критерия Фишера и сравните получившиеся значения с двумя другими подходами, описанными выше.

Точный критерий Фишера удобно вычислять с помощью функции scipy.stats.fisher_exact, которая принимает на вход таблицу сопряженности 2x2. 

In [59]:
fisher_cross_table = pd.DataFrame(columns=['state_1', 'state_2', 'p-value'])

In [60]:
i=0
for state1, state2 in combinations(state_churn.index, 2):
    fisher_ = state_churn.loc[[state1, state2],:]
    fisher_cross_table.loc[i, 'state_1'] = state1
    fisher_cross_table.loc[i, 'state_2'] = state2
    fisher_cross_table.loc[i, 'p-value'] = stats.fisher_exact(fisher_.values)[1]
    i+=1

In [62]:
fisher_cross_table[fisher_cross_table['p-value']<0.05]['p-value'].count()

10

*Точный критерий Фишера на наших данных дает значения достигаемого уровня значимости в среднем меньшие, чем xи-квадрат с поправкой Йетса*


*Точный критерий Фишера на наших данных дает значения достигаемого уровня значимости в среднем значительно большие, чем xи-квадрат без поправки*


*Точный критерий Фишера всегда лучше, чем критерий xи-квадрат, потому что не использует аппроксимацию дискретного распределения непрерывным. Однако при увеличении размера выборки его преимущества по сравнению с критерем xи-квадрат уменьшаются, в пределе достигая нуля*

Давайте попробуем применить полученные знания о разных видах корреляции и ее применимости на практике. 

Рассмотрим пару признаков day_calls и  mes_estim. Посчитайте корреляцию Пирсона между этими признаками на всех данных, ее значимость. 

Отметьте все верные утверждения. 

In [66]:
stats.pearsonr(data['day_calls'], data['mes_estim'])

(-0.05179435058757264, 0.0027798836869738384)

*Корреляция Пирсона имеет отрицательный знак, и отличие корреляции от нуля на уровне доверия 0.05 значимо*

In [70]:
stats.spearmanr(data['day_calls'], data['mes_estim'])

SpearmanrResult(correlation=0.043349880533927444, pvalue=0.012317367189170541)

*Корреляция Спирмена имеет положительный знак, и отличие корреляции от нуля на уровне доверия 0.05 значимо*

*Посчитанные корреляции и их значимости говорят лишь о том, что необходимо взглянуть на данные глазами и попытаться понять, что приводит к таким (противоречивым?) результатам.*

Посчитайте значение коэффицента корреляции Крамера между двумя признаками:  штатом (state) и оттоком пользователей (churn) для всех пользователей, которые находились в контрольной группе (treatment=1).  Что можно сказать о достигаемом уровне значимости при проверке гипотезы о равенство нулю этого коэффициента?

In [71]:
def cramers_stat(confusion_matrix):
    chi2 = stats.chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum()
    return np.sqrt(chi2 / (n*(min(confusion_matrix.shape)-1)))

In [73]:
cramers_stat(state_churn.values)

0.2003932150203332

In [74]:
stats.chi2_contingency(state_churn.values)[1]

0.7097590042778473

*Для вычисления коэффициента Крамера используется значение статистики xи-квадрат, на которую мы не можем положиться применительно к нашим данным*

В этой части задания вам нужно будет самостоятельно решить, с помощью каких методов можно провести анализ эффективности удержания (churn) с помощью раличных методов (treatment = 0, treatment = 2) относительно контрольной группы пользователей (treatment = 1). 

Что можно сказать об этих двух методах (treatment = 0, treatment = 2)? Одинаковы ли они с точки зрения эффективности? Каким бы методом вы бы посоветовали воспользоваться компании?

Не забудьте про поправку на множественную проверку!  И не пользуйтесь односторонними альтернативами, поскольку вы не знаете, к каким действительно последствиям приводят тестируемые методы  (treatment = 0, treatment = 2) !

In [168]:
treat_0_group = data[data["treatment"] == 0]
treat_0_group_1 = treat_0_group[treat_0_group["churn"] == "True."]
treat_2_group = data[data["treatment"] == 2]
treat_2_group_1 = treat_2_group[treat_2_group["churn"] == "True."]
control_group = data[data["treatment"] == 1]
control_group_1 = control_group[control_group["churn"] == "True."]

In [170]:
treat_0_count = float(len(treat_0_group))
treat_0_count_1 = float(len(treat_0_group_1))
treat_2_count = float(len(treat_2_group))
treat_2_count_1 = float(len(treat_2_group_1))
control_group_count = float(len(control_group))
control_group_count_1 = float(len(control_group_1))

In [173]:
proportion_confint(treat_0_count_1, treat_0_count)

(0.12509189855388805, 0.16617023736844203)

In [174]:
proportion_confint(treat_2_count_1, treat_2_count)

(0.10558846307607082, 0.14463819150235163)

In [175]:
proportion_confint(control_group_count_1, control_group_count)

(0.14216797720187604, 0.18599975297132357)

In [144]:
def proportions_diff_z_stat_ind(sample1, sample2):
    n1 = len(sample1)
    n2 = len(sample2)
    
    p1 = float(sum(sample1)) / n1
    p2 = float(sum(sample2)) / n2 
    P = float(p1*n1 + p2*n2) / (n1 + n2)
    
    return (p1 - p2) / np.sqrt(P * (1 - P) * (1. / n1 + 1. / n2))

def proportions_diff_z_test(sample1, sample2, alternative = "two-sided"):
    if alternative not in ("two-sided", "less", "greater"):
        raise ValueError("alternative not recognized\n"
                         "should be \"two-sided\", \"less\" or \"greater\"")
    z_stat = proportions_diff_z_stat_ind(sample1, sample2)
    p_value = 0
    if alternative == 'two-sided':
        p_value = 2 * (1 - stats.norm.cdf(np.abs(z_stat)))
    elif alternative == 'less':
        p_value = stats.norm.cdf(z_stat)
    elif alternative == 'greater':
        p_value = 1 - stats.norm.cdf(z_stat)

    return (z_stat,p_value)

In [188]:
treat_0_group_ = list(map(lambda x: 1. if x == "True." else 0., treat_0_group["churn"].values))
treat_2_group_ = list(map(lambda x: 1. if x == "True." else 0., treat_2_group["churn"].values))
control_group_ = list(map(lambda x: 1. if x == "True." else 0., control_group["churn"].values))

In [197]:
test_0_control = proportions_diff_z_test(treat_0_group_, control_group_)
test_0_control

(-1.2046688551240339, 0.2283311639045107)

In [198]:
test_2_control = proportions_diff_z_test(treat_2_group_, control_group_)
test_2_control

(-2.599054820975787, 0.009348084294451109)

In [199]:
test_0_2 = proportions_diff_z_test(treat_0_group_, treat_2_group_)
test_0_2

(1.4171992240305467, 0.1564246886050802)

In [195]:
p_values = multipletests([test_0_control[1], test_2_control[1], test_0_2[1]],
    alpha = 0.05,
    method = "fdr_bh")[1]

In [196]:
p_values

array([0.22833116, 0.02804425, 0.22833116])

*treatment = 2 статистически значимо отличается от контрольной группы treatment = 1*


*Отличие между treatment = 0 и treatment = 2 относительно влияния на уровень churn статистически незначимо*